In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
with open('Naruto.txt', 'r', encoding='utf-8') as f:
    txt_content = f.read()

In [ ]:
! pip install PyPDF2
! pip install python-docx

In [ ]:
import PyPDF2

pdf_content = ""
with open('Sakura.pdf', 'rb') as f:
    reader = PyPDF2.PdfReader(f)
    for page in reader.pages:
        pdf_content += page.extract_text() or ""

In [ ]:
import docx

def read_docx(filename):
    doc = docx.Document(filename)
    return "\n".join([para.text for para in doc.paragraphs])

docx_content = read_docx('Sasuke.docx')

In [ ]:
text = txt_content + pdf_content + docx_content

In [ ]:
chars = sorted(list(set(text)))
char_to_index = {char: i for i, char in enumerate(chars)}
index_to_char = {i: char for i, char in enumerate(chars)}

In [ ]:
seq_length = 3
sequences = []
labels = []

In [ ]:
for i in range(len(text)-seq_length):
  seq = text[i:i + seq_length]
  label = text[i + seq_length]
  sequences.append([char_to_index[char] for char in seq])
  labels.append(char_to_index[label])

In [ ]:
x = np.array(sequences)
y = np.array(labels)

In [ ]:
x_tensor = torch.from_numpy(x)
y_tensor = torch.from_numpy(y)

In [ ]:
x_one_hot = torch.nn.functional.one_hot(x_tensor, num_classes = len(chars)).float()

In [ ]:
dataset = TensorDataset(x_one_hot, y_tensor)
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class CharLSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
     super(CharLSTM, self).__init__()
     self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
     self.fc = nn.Linear(hidden_size, num_classes)
  def forward(self, x):
     out, _ = self.lstm(x)
     out = out[:, -1, :]
     out = self.fc(out)
     return out


In [ ]:
input_size = len(chars)
hidden_size = 160
num_layers = 6
num_classes = len(chars)
num_epochs = 300
learning_rate = 0.001

In [ ]:
model = CharLSTM(input_size, hidden_size, num_layers, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(num_epochs):
  total_loss = 0
  for batch_x, batch_y in dataloader:
    optimizer.zero_grad()
    outputs = model(batch_x)
    loss = criterion(outputs, batch_y)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  avg_loss = total_loss/len(dataloader)
  print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss: .4f}")

In [ ]:
def generate_text(model, start_seq, length, char_to_index, index_to_char):
  model.eval()
  seq = [char_to_index[c] for c in start_seq]
  generated = start_seq
  for _ in range(length):
    x = torch.tensor([seq[-seq_length:]])
    x_onehot = torch.nn.functional.one_hot(x, num_classes=len(chars)).float()
    with torch.no_grad():
      out = model(x_onehot)
      pred = out.argmax(dim=1).item()
    generated += index_to_char[pred]
    seq.append(pred)
  return generated

In [ ]:
print("Generated Text: ")
print(generate_text(model, "Once", 100, char_to_index, index_to_char))

## 📦 Exporting Model and Vocab

In [ ]:
import torch
import pickle

# Save model
torch.save(model.state_dict(), "text_generator_model.pth")

# Save vocab and mappings
with open("vocab.pkl", "wb") as f:
    pickle.dump(vocab, f)

with open("char_to_idx.pkl", "wb") as f:
    pickle.dump(char_to_idx, f)

with open("idx_to_char.pkl", "wb") as f:
    pickle.dump(idx_to_char, f)

print("✅ Model and mappings exported successfully.")


## 🧠 Text Generation Function

In [ ]:
def generate_text(start_seq="The", length=300):
    model.eval()
    input_seq = torch.tensor([char_to_idx[c] for c in start_seq], dtype=torch.long).unsqueeze(0)
    hidden = None
    result = start_seq

    for _ in range(length):
        with torch.no_grad():
            output, hidden = model(input_seq, hidden)
            probs = torch.nn.functional.softmax(output[-1], dim=0).cpu()
            next_char_idx = torch.multinomial(probs, 1).item()
            next_char = idx_to_char[next_char_idx]

        result += next_char
        input_seq = torch.tensor([[next_char_idx]], dtype=torch.long)

    return result

# Example usage
print(generate_text("The future of AI", 500))


## 🧪 CLI Support (Optional)

In [ ]:
if __name__ == "__main__":
    prompt = input("Enter a blog title or starting text: ")
    print("\nGenerated Blog:\n")
    print(generate_text(prompt, 500))
